In [1]:
import sys
sys.executable

'/Library/Frameworks/Python.framework/Versions/3.8/bin/python3'

In [3]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns 


### Importing the relevant ML libraries 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


from sklearn.metrics import accuracy_score,roc_curve,auc,recall_score,f1_score,precision_score,classification_report,confusion_matrix,auc

### Some cosmetics add-ons
plt.style.use('fivethirtyeight')
%matplotlib inline

Read csv file

In [4]:
df = pd.read_csv('annual_data/nba_season_2014-2020.csv')

In [5]:
df.drop(df[(df.GAME_PLAYED_x == 1) | (df.GAME_PLAYED_y == 1)].index, inplace = True)

Drop rows of all the first games of every team

Extract first 41 games (mid season) of every team

In [6]:
first_half = df[(df['GAME_PLAYED_x'] <= 41) | (df['GAME_PLAYED_y'] <= 41 )]
first_half

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,OFFRATE_y,DEFRATE_y,ELO_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,DIS_OFFRATE_y,DIS_DEFRATE_y,DIS_ELO_y
16,22014,1610612765,DET,Detroit Pistons,21400017,2014-10-30,DET @ MIN,0,241,91,...,1.018145,1.058468,1490.000000,22.000000,5.000000,11.000000,-9.000000,0.216928,0.155831,0.000000
17,22014,1610612762,UTA,Utah Jazz,21400019,2014-10-30,UTA @ DAL,0,241,102,...,1.154734,1.166282,1490.000000,7.000000,1.000000,-4.000000,3.000000,0.095509,-1.248297,-0.287744
18,22014,1610612764,WAS,Washington Wizards,21400016,2014-10-30,WAS @ ORL,1,240,105,...,0.889831,1.069915,1490.000000,-11.000000,-3.000000,9.000000,11.000000,-0.147287,-0.098207,0.000000
20,22014,1610612758,SAC,Sacramento Kings,21400025,2014-10-31,SAC vs. POR,1,240,103,...,1.142241,0.959052,1510.000000,29.000000,10.000000,-2.000000,-6.000000,0.394669,0.036722,20.000000
21,22014,1610612759,SAS,San Antonio Spurs,21400024,2014-10-31,SAS @ PHX,0,239,89,...,1.263270,2.006369,1509.712256,18.000000,0.000000,0.000000,2.000000,0.107663,0.862205,-0.287744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7285,22020,1610612759,SAS,San Antonio Spurs,22000176,2021-01-14,SAS vs. HOU,0,238,105,...,1.087948,1.375679,1476.427355,0.151515,-2.434343,-0.979798,-2.141414,0.007941,0.451977,-33.707151
7286,22020,1610612755,PHI,Philadelphia 76ers,22000174,2021-01-14,PHI vs. MIA,1,239,125,...,1.079509,1.466279,1490.196640,-2.083333,0.944444,-2.750000,-0.888889,-0.010163,0.664037,-38.782458
7287,22020,1610612757,POR,Portland Trail Blazers,22000178,2021-01-14,POR vs. IND,0,240,87,...,1.116127,1.002832,1524.393682,-4.000000,3.636364,-0.363636,-1.000000,-0.024807,-0.201049,-2.669028
7288,22020,1610612743,DEN,Denver Nuggets,22000177,2021-01-14,DEN vs. GSW,1,240,114,...,1.074953,1.196395,1516.631270,-3.154545,-2.381818,-2.000000,4.172727,-0.078894,0.024534,22.314357


Define the DataFrame to be processed

In [7]:
clean_df = first_half

In [54]:
y = clean_df['PLUS_MINUS_x']
x = clean_df[['DIS_ELO_x','HOME_COURT_x','DIS_OFFRATE_x','DIS_DEFRATE_x','DIS_PTS_x','DIS_AST_x','DIS_OREB_x','DIS_DREB_x']]

model = sm.OLS(y,x)
results = model.fit()
results.params  

DIS_ELO_x         0.044024
HOME_COURT_x      2.413302
DIS_OFFRATE_x    29.630766
DIS_DEFRATE_x    -0.262760
DIS_PTS_x        -0.131305
DIS_AST_x         0.139873
DIS_OREB_x       -0.001489
DIS_DREB_x        0.284768
dtype: float64

In [55]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           PLUS_MINUS_x   R-squared (uncentered):                   0.141
Model:                            OLS   Adj. R-squared (uncentered):              0.140
Method:                 Least Squares   F-statistic:                              78.10
Date:                Mon, 08 Feb 2021   Prob (F-statistic):                   8.35e-120
Time:                        15:40:44   Log-Likelihood:                         -15154.
No. Observations:                3801   AIC:                                  3.032e+04
Df Residuals:                    3793   BIC:                                  3.037e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
DIS_ELO_x         0.0440      0.003     12.764      0.000       0.037       0.051
HOME_COURT_x      2.4133      0.303      7.972      0.000       1.820       3.007
DIS_OFFRATE_x    29.6308      4.829      6.136      0.000      20.163      39.099
DIS_DEFRATE_x    -0.2628      1.092     -0.241      0.810      -2.404       1.879
DIS_PTS_x        -0.1313      0.033     -3.994      0.000      -0.196      -0.067
DIS_AST_x         0.1399      0.074      1.890      0.059      -0.005       0.285
DIS_OREB_x       -0.0015      0.096     -0.016      0.988      -0.189       0.186
DIS_DREB_x        0.2848      0.073      3.908      0.000       0.142       0.428
==============================================================================
Omnibus:                       17.813   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.358
Skew:                          -0.000   Prob(JB):                     5.14e-06
Kurtosis:                       3.392   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Create a list of features that are deemed significant (p-value < 0.05)

In [66]:
features_list = []
for i in range(len(x.keys())):
    if results.pvalues[i] <= 0.05:
        features_list.append(model.exog_names[i])

features_list

['DIS_ELO_x', 'HOME_COURT_x', 'DIS_OFFRATE_x', 'DIS_PTS_x', 'DIS_DREB_x']

Results: x1(DIS_ELO), x2(HOME_COURT) and x3(DIS_OFFRATE) are significant variables
Chosen model is to be fitted the 3 significant x variables
Resulting equation would be used to predict outcome of a game:
    - postiive --> team 1 wins
    - negative --> team 2 wins

Part 2: Using regression based on the 3 significant factors to predict games in the 2nd half of season

Extract 2nd half of season from DataFrame to predict outcomes of all games

In [68]:
second_half = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41 )]
second_half

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,OFFRATE_y,DEFRATE_y,ELO_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,DIS_OFFRATE_y,DIS_DEFRATE_y,DIS_ELO_y
610,22014,1610612754,IND,Indiana Pacers,21400617,2015-01-19,IND @ HOU,0,240,98,...,1.049528,1.044530,1580.810254,7.534262,0.261324,1.172474,-3.160859,0.054917,0.067137,159.771236
611,22014,1610612741,CHI,Chicago Bulls,21400620,2015-01-19,CHI @ CLE,0,242,94,...,1.074796,1.110475,1491.954552,-1.650987,-0.221835,-0.904762,-3.305459,-0.003931,0.047061,-75.604759
613,22014,1610612747,LAL,Los Angeles Lakers,21400622,2015-01-19,LAL @ PHX,0,240,100,...,1.084334,0.903411,1533.204714,6.633566,0.128920,-0.772938,0.178862,0.043416,-0.069226,126.475792
618,22014,1610612737,ATL,Atlanta Hawks,21400613,2015-01-19,ATL vs. DET,1,240,93,...,1.029925,0.928036,1483.404867,-5.439024,-4.707317,4.195122,0.268293,-0.061377,-0.178982,-199.041129
622,22014,1610612759,SAS,San Antonio Spurs,21400624,2015-01-20,SAS @ DEN,1,241,109,...,1.033295,1.065748,1476.098464,-0.077816,-3.425087,2.653891,-0.610918,-0.028010,0.094043,-85.715502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7116,22019,1610612757,POR,Portland Trail Blazers,21900966,2020-03-10,POR vs. PHX,1,240,121,...,1.102937,1.165035,1445.947118,-0.851202,6.987500,-0.295192,-2.088221,-0.008225,0.131726,-14.125912
7117,22019,1610612737,ATL,Atlanta Hawks,21900969,2020-03-11,ATL vs. NYK,0,265,131,...,1.049928,1.134572,1395.926751,-6.131235,-2.016550,2.350583,1.035664,-0.005747,0.190678,-11.700099
7118,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,1,240,113,...,1.114136,1.213762,1590.753386,-5.799716,2.078125,0.284091,-2.879735,-0.041028,0.141893,63.645667
7119,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,0,241,106,...,1.097332,1.066915,1542.411779,2.097837,1.782212,0.497115,3.031971,0.019476,0.026163,194.443829


Define variables

In [69]:
# for splitting by halfs
X_train = first_half[features_list]
X_test = second_half[features_list]
y_train = first_half['PLUS_MINUS_x']
y_test = second_half['WL_x']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3801, 5), (3373, 5), (3801,), (3373,))

Modelling

In [70]:
m = LinearRegression()
m.fit(X_train, y_train)
predictions = m.predict(X_test)

predictions

array([ -9.56586429,   1.9059188 ,  -8.74752754, ...,   1.00837714,
       -12.37295349,  11.73778424])

Converting estimated point difference to binary outcome:

if y > 0 --> W --> 1

else --> L --> 0

In [71]:
for i, v in enumerate(predictions):
    if v > 0:
        predictions[i] = 1
    else:
        predictions[i] = 0

predictions

array([0., 1., 0., ..., 1., 0., 1.])

Check performance of model

In [72]:
acc = accuracy_score(y_test,predictions)
precision = precision_score(y_test,predictions) 
recall = recall_score(y_test,predictions) 
f1 = f1_score(y_test,predictions)

acc, precision, recall, f1

(0.6629113548769642,
 0.6619469026548672,
 0.6654804270462633,
 0.6637089618456078)

Part 3: Using regression based on all factors to predict games in the 2nd half of season

Redefine model variables

In [17]:
features_list = ['DIS_ELO_x', 'HOME_COURT_x', 'DIS_OFFRATE_x', 'DIS_DEFRATE_x', 'DIS_PTS_x', 'DIS_AST_x', 'DIS_OREB_x', 'DIS_DREB_x']

In [18]:
X_train = first_half[features_list]
X_test = second_half[features_list]
y_train = first_half['PLUS_MINUS_x']
y_test = second_half['WL_x']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3801, 8), (3373, 8), (3801,), (3373,))

Modelling

In [19]:
m = LinearRegression()
m.fit(X_train, y_train)
predictions = m.predict(X_test)

predictions

array([ -9.19861536,   1.80700534,  -8.45463955, ...,   0.63467252,
       -12.37360842,  11.60434196])

Converting estimated point difference to binary outcome

In [20]:
for i, v in enumerate(predictions):
    if v > 0:
        predictions[i] = 1
    else:
        predictions[i] = 0

predictions

array([0., 1., 0., ..., 1., 0., 1.])

Check performance of model

In [21]:
acc = accuracy_score(y_test,predictions)
precision = precision_score(y_test,predictions) 
recall = recall_score(y_test,predictions) 
f1 = f1_score(y_test,predictions)

acc, precision, recall, f1

(0.6670619626445301, 0.6668642560758743, 0.6672597864768683, 0.66706196264453)

Part 4: Check accuracy of linear regression model for all seasons, features selected ONLY IF it is significant.

In [73]:
s2014 = pd.read_csv('annual_data/nba_season_2014_data.csv')
s2015 = pd.read_csv('annual_data/nba_season_2015_data.csv')
s2016 = pd.read_csv('annual_data/nba_season_2016_data.csv')
s2017 = pd.read_csv('annual_data/nba_season_2017_data.csv')
s2018 = pd.read_csv('annual_data/nba_season_2018_data.csv')
s2019 = pd.read_csv('annual_data/nba_season_2019_data.csv')

In [75]:
season = 2014
for df in (s2014, s2015, s2016, s2017, s2018, s2019):
    df.drop(df[(df.GAME_PLAYED_x == 1) | (df.GAME_PLAYED_y == 1)].index, inplace = True)

    first_half = df[(df['GAME_PLAYED_x'] <= 41) | (df['GAME_PLAYED_y'] <= 41 )]

    y = df['PLUS_MINUS_x']
    x = df[['DIS_ELO_x','HOME_COURT_x','DIS_OFFRATE_x','DIS_DEFRATE_x','DIS_PTS_x','DIS_AST_x','DIS_OREB_x','DIS_DREB_x']]

    model = sm.OLS(y,x)
    results = model.fit()

    features_list = []
    for i in range(len(x.keys())):
        if results.pvalues[i] <= 0.05:
            features_list.append(model.exog_names[i])

    second_half = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41 )]

    X_train = first_half[features_list]
    X_test = second_half[features_list]
    y_train = first_half['PLUS_MINUS_x']
    y_test = second_half['WL_x']

    m = LinearRegression()
    m.fit(X_train, y_train)
    predictions = m.predict(X_test)

    for i, v in enumerate(predictions):
        if v > 0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    acc = accuracy_score(y_test,predictions)

    print('Accuracy Score for Linear Regression is {:.5f} in season {}'.format(acc,season))

    season += 1

Accuracy Score for Linear Regression is 0.68482 in season 2014
Accuracy Score for Linear Regression is 0.71287 in season 2015
Accuracy Score for Linear Regression is 0.62975 in season 2016
Accuracy Score for Linear Regression is 0.68046 in season 2017
Accuracy Score for Linear Regression is 0.66667 in season 2018
Accuracy Score for Linear Regression is 0.64162 in season 2019


Part 5: Check accuracy of linear regression model for all seasons, using all features.

In [76]:
season = 2014
for df in (s2014, s2015, s2016, s2017, s2018, s2019):
    df.drop(df[(df.GAME_PLAYED_x == 1) | (df.GAME_PLAYED_y == 1)].index, inplace = True)

    first_half = df[(df['GAME_PLAYED_x'] <= 41) | (df['GAME_PLAYED_y'] <= 41 )]

    y = df['PLUS_MINUS_x']
    x = df[['DIS_ELO_x','HOME_COURT_x','DIS_OFFRATE_x','DIS_DEFRATE_x','DIS_PTS_x','DIS_AST_x','DIS_OREB_x','DIS_DREB_x']]

    model = sm.OLS(y,x)
    results = model.fit()

    features_list = []
    for i in range(len(x.keys())):
        features_list.append(model.exog_names[i])

    second_half = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41 )]

    X_train = first_half[features_list]
    X_test = second_half[features_list]
    y_train = first_half['PLUS_MINUS_x']
    y_test = second_half['WL_x']

    m = LinearRegression()
    m.fit(X_train, y_train)
    predictions = m.predict(X_test)

    for i, v in enumerate(predictions):
        if v > 0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    acc = accuracy_score(y_test,predictions)

    print('Accuracy Score for Linear Regression is {:.5f} in season {}'.format(acc,season))

    season += 1

Accuracy Score for Linear Regression is 0.67657 in season 2014
Accuracy Score for Linear Regression is 0.70132 in season 2015
Accuracy Score for Linear Regression is 0.61983 in season 2016
Accuracy Score for Linear Regression is 0.66391 in season 2017
Accuracy Score for Linear Regression is 0.66172 in season 2018
Accuracy Score for Linear Regression is 0.64740 in season 2019


# Interaction Terms

In [26]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns 

### Importing the relevant ML libraries 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.regression import linear_model

from sklearn.metrics import accuracy_score,roc_curve,auc,recall_score,f1_score,precision_score,classification_report,confusion_matrix,auc

### Some cosmetics add-ons
plt.style.use('fivethirtyeight')
%matplotlib inline

In [109]:
s2014 = pd.read_csv('annual_data/nba_season_2014_data.csv')

In [110]:
df.drop(df[(df.GAME_PLAYED_x == 1) | (df.GAME_PLAYED_y == 1)].index, inplace = True)

In [111]:
first_half = df[(df['GAME_PLAYED_x'] <= 41) | (df['GAME_PLAYED_y'] <= 41 )]
second_half = df[(df['GAME_PLAYED_x'] > 41) & (df['GAME_PLAYED_y'] > 41 )]

In [112]:
y = first_half['PLUS_MINUS_x']
X = first_half[['DIS_ELO_x','HOME_COURT_x','DIS_OFFRATE_x','DIS_DEFRATE_x','DIS_PTS_x','DIS_AST_x','DIS_OREB_x','DIS_DREB_x']]

In [113]:
X_interaction = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(X)
test = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit(X)
column_names = test.get_feature_names()

In [114]:
interaction_df = pd.DataFrame(X_interaction, columns = column_names)

In [116]:
y = y.reset_index()
y = y['PLUS_MINUS_x']

In [118]:
model = sm.OLS(y,interaction_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           PLUS_MINUS_x   R-squared (uncentered):                   0.155
Model:                            OLS   Adj. R-squared (uncentered):              0.147
Method:                 Least Squares   F-statistic:                              19.16
Date:                Sat, 06 Mar 2021   Prob (F-statistic):                   7.46e-111
Time:                        21:44:07   Log-Likelihood:                         -15124.
No. Observations:                3801   AIC:                                  3.032e+04
Df Residuals:                    3765   BIC:                                  3.054e+04
Df Model:                          36                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x0             0.0449      0.005      9.091      0.000       0.035       0.055
x1             2.8983      0.337      8.606      0.000       2.238       3.559
x2            31.7789      8.290      3.833      0.000      15.526      48.032
x3             0.6922      1.643      0.421      0.674      -2.529       3.914
x4            -0.0514      0.062     -0.832      0.405      -0.172       0.070
x5             0.0717      0.102      0.705      0.481      -0.128       0.271
x6             0.0178      0.137      0.130      0.896      -0.250       0.286
x7             0.1334      0.107      1.243      0.214      -0.077       0.344
x0 x1         -0.0065      0.007     -0.941      0.347      -0.020       0.007
x0 x2         -0.0979      0.068     -1.438      0.150      -0.231       0.036
x0 x3         -0.0173      0.023     -0.738      0.460      -0.063       0.029
x0 x4          0.0005      0.001      0.633      0.527      -0.001       0.002
x0 x5          0.0011      0.001      0.986      0.324      -0.001       0.003
x0 x6         -0.0015      0.002     -0.855      0.392      -0.005       0.002
x0 x7         -0.0018      0.001     -1.684      0.092      -0.004       0.000
x1 x2          1.9211      9.814      0.196      0.845     -17.321      21.163
x1 x3         -0.3040      2.227     -0.136      0.891      -4.671       4.063
x1 x4         -0.1350      0.067     -2.025      0.043      -0.266      -0.004
x1 x5          0.1835      0.149      1.233      0.218      -0.108       0.475
x1 x6          0.0001      0.192      0.001      1.000      -0.377       0.377
x1 x7          0.3006      0.147      2.050      0.040       0.013       0.588
x2 x3        -26.4673     14.466     -1.830      0.067     -54.830       1.895
x2 x4         -0.0640      0.392     -0.163      0.870      -0.832       0.704
x2 x5         -1.1726      1.210     -0.969      0.332      -3.545       1.199
x2 x6          0.9737      1.440      0.676      0.499      -1.849       3.796
x2 x7         -0.0397      0.801     -0.050      0.961      -1.610       1.531
x3 x4          0.0470      0.122      0.384      0.701      -0.193       0.287
x3 x5         -0.4704      0.319     -1.474      0.141      -1.096       0.155
x3 x6         -1.0439      0.355     -2.944      0.003      -1.739      -0.349
x3 x7          0.4204      0.246      1.712      0.087      -0.061       0.902
x4 x5         -0.0054      0.004     -1.215      0.224      -0.014       0.003
x4 x6         -0.0021      0.009     -0.241      0.809      -0.019       0.015
x4 x7          0.0041      0.003      1.383      0.167      -0.002       0.010
x5 x6          0.0633      0.027      2.308      0.021       0.010       0.117
x5 x7         -0.0057      0.015     -0.368      0.713      -

In [119]:
features_list = []
for i in range(len(interaction_df.keys())):
    if result.pvalues[i] <= 0.05:
        features_list.append(model.exog_names[i])

features_list

['x0', 'x1', 'x2', 'x1 x4', 'x1 x7', 'x3 x6', 'x5 x6']

In [120]:
final_interaction_features = ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x1 x4', 'x1 x7', 'x3 x6', 'x5 x6']

In [121]:
interaction_second_half = second_half[['DIS_ELO_x','HOME_COURT_x','DIS_OFFRATE_x','DIS_DEFRATE_x','DIS_PTS_x','DIS_AST_x','DIS_OREB_x','DIS_DREB_x']]
interaction_second_half = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(interaction_second_half)
second_half_fit = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit(X)
column_names = second_half_fit.get_feature_names()
interaction_second_half_df = pd.DataFrame(interaction_second_half, columns = column_names)

In [122]:
X_train = interaction_df[final_interaction_features]
X_test = interaction_second_half_df[final_interaction_features]
y_train = first_half['PLUS_MINUS_x']
y_test = second_half['WL_x']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3801, 12), (3373, 12), (3801,), (3373,))

In [127]:
m = LinearRegression()
m.fit(X_train, y_train)
predictions = m.predict(X_test)

In [124]:
for i, v in enumerate(predictions):
    if v > 0:
        predictions[i] = 1
    else:
        predictions[i] = 0

In [125]:
acc = accuracy_score(y_test,predictions)
precision = precision_score(y_test,predictions) 
recall = recall_score(y_test,predictions) 
f1 = f1_score(y_test,predictions)

acc, precision, recall, f1

(0.6658760747109398, 0.6650915534554046, 0.66785290628707, 0.6664693696359869)

This is the respective acc, prec, rec, f1 score without interaction terms
(0.6629113548769642,
 0.6619469026548672,
 0.6654804270462633,
 0.6637089618456078)